This is a notebook and associated template that allows you to filter output from entire cells by adding the `tags: ['hidden']` value to cell metadata.

It is based on http://www.damian.oquanta.info/posts/mimic-the-ipython-notebook-cell-execution.html which focused instead on just filtering output.  

If you can see a bunch of checked checkmarks on the left, then don't fear, there's a lot more text here that will explain what's going on.

We'll do this entirely with CSS rather than relying on javascript. 

This is at the cost of uniform animation smoothness. But it's still useful to see how to do it.

In [2]:
!sed -n 1,2p toggle_hidden.tpl

{%- extends 'full.tpl' -%}



This declares which template we are extending, `full.tpl` which can be found in `nbconvert.templates.html`.

Next, we need to declare the wrapper div that will hold the original contents of the cell (invoked via `{{super()}}` and our checkbox that we'll be put off to the side of it. 

We'll be specifying that kind of placement information through the CSS calls, right now we just need to describe how we're going to change the DOM structure so we can manipulate it safely using CSS selectors.

Note that the checkboxes default to being checked.

In [9]:
!sed -n 3,8p toggle_hidden.tpl

{%- block any_cell -%}
{%- if 'hidden' in cell.metadata.get("tags",[]) -%}
    <div class="hidden_cell">
    <input class="hide_cell" type="checkbox" checked></input>
        {{super() }}
    </div>


We add a div to wrap the cell and the checkbox. 

We had to *rewrite* the `output_group` block to encompass it with a new `output_hidden`-classed `div`, which we would use in a simple **Javascript** function later.

Second, we needed to write that little **Javascript** snippet:

In [6]:
!sed -n 25,29p toggle_hidden.tpl

{%- block header -%}
{{ super() }}

<style type="text/css">
div.hidden_cell > div.cell{


Here, we used the `.slideToggle()` method to display or hide the `output_hidden`-classed `div` elements (with a nice sliding motion) when the user makes a click on the input cells. We was using **jQuery**, so we needed to load **jQquery**, ie. using the following line:

`<script src="//ajax.googleapis.com/ajax/libs/jquery/1.10.2/jquery.min.js"></script>`

Finally, we added some css to make the `output_hidden-classed div` not visible (hidden by default):

In [3]:
!sed -n 18,21p /media/datos/Ejemplos/toggle.tpl

.output_hidden {
  display: none;
  margin-top: 5px;
}


If you put all these snippets together, you will get the following template:

In [4]:
!cat -n /media/datos/Ejemplos/toggle.tpl

     1	{%- extends 'html_full.tpl' -%}
     2	
     3	{% block output_group %}
     4	<div class="output_hidden">
     5	{{ super() }}
     6	</div>
     7	{% endblock output_group %}
     8	
     9	{%- block header -%}
    10	{{ super() }}
    11	
    12	<script src="//ajax.googleapis.com/ajax/libs/jquery/1.10.2/jquery.min.js"></script>
    13	
    14	<style type="text/css">
    15	div.output_wrapper {
    16	  margin-top: 0px;
    17	}
    18	.output_hidden {
    19	  display: none;
    20	  margin-top: 5px;
    21	}
    22	</style>
    23	
    24	<script>
    25	$(document).ready(function(){
    26	  $(".input").click(function(){
    27	      $(this).next('.output_hidden').slideToggle();
    28	  });
    29	})
    30	</script>
    31	{%- endblock header -%}
    32	


To use it, just add `--template toggle` to your `ipython nbconvert` call, ie:

`ipython nbconvert your_notebook.ipynb --to html --template toggle --post serve`

or add the `toggle.tpl` file to you **IPython** nbconvert config file.

If you are reading this line, you probably have already experienced the demo in this post (btw, I like to do demos explaining the tools with the tools themselves).

Obviously, I quickly implemented this functionality inside **Nikola** and, as a consequence, in **Nikola** master, you now have the possibility to pass an **IPython** config `dict` to your **Nikola** `conf.py` file, making the *ipynb* plugin for **Nikola** more powerful than before ;-)

To conclude, the possibility to mimic the **IPython** notebook execution is very useful for introductory courses and to publish content in a shorter but more interactive way.

Hope you enjoy it!

Damián